# TheBrain API Playground
This Jupyter Notebook contains a set of cells that interact with TheBrain API. The first cell imports necessary modules and reads in API keys and target brain information from text files. The following cells define functions to interact with the API, including listing brains, scanning brains, listing types and tags, searching for thoughts, and retrieving named items. Each of these functions is demonstrated in a separate cell.

This Version of the playground only supports read-only "get" calls.

In [ ]:
import os
import requests
from json import JSONDecodeError

api_path = 'APIKey.txt'
 # The following ID is a read-only public brain for testing. The Bard.
 # Only read-only calls will work on TheBard
 # Change this or add a text file called 
 # `target.txt` that just contains the ID of your brain.
target_brain='1c9d0bdb-8e55-43b2-ac53-959e4c2c8864'
 # This code doesn't contain my API key. :) Either add your API key below or 
 # create a file called `APIKey.txt` that just contains the key.
apiKey = ""
homeThought = ""

try:
    # Check whether the file exists
    if os.path.exists(api_path):
        # Open the file as f
        # Don't forget 'r' - read mode
        with open(api_path, 'r') as file:
            # Read the contents of the file into a string variable
            apiKey = file.read()
        print('API Key Imported. Not shown for security reasons')
    else:
        print('APIKey.txt does not exist. Please create it and add your API key.')
except:
    print('Failed to get an API key, please create APIKey.txt in your folder.')
    

# I have a text file with my main target brain. This reads this in and sets the target_Brain.
try:
    if os.path.exists('target.txt'):
        with open( 'target.txt', 'r') as file:
            # Read the contents of the file into a string variable
            target_brain = file.read()
finally:
    print('Target Brain: ' + target_brain)

# Define the request parameters
baseUrl = "https://api.bra.in/"
headers = {"Content-Type": "application/json", 'Authorization': f'Bearer {apiKey}', 'Krang-Version': '2020-06-10'}
homeThought = ''

In [ ]:
from urllib.parse import quote

# This is the main "switchboard" get function.
def get(type,data=None, search="", results=30, target=None, nameOnly=True):
    if target is None:
        target = homeThought
    get_headers = {"Content-Type": "application/json", 'Authorization': f'Bearer {apiKey}', 'Krang-Version': '2020-06-10'}
    addURL = ""
    search = quote(search)
    """Send a get request to the API"""
    if type == "brainscan":
        if verbose: print("You chose option Brainscan.")
        addURL = f"brains/{target_brain}"
    elif type == "brainlist":
        if verbose: print("You chose option Brainlist.")
        addURL = "brains"
    elif type == "types":
        addURL = f'thoughts/{target_brain}/types'
        if verbose: print("You chose types.")
    elif type == "tags":
        addURL = f'thoughts/{target_brain}/tags'
        if verbose: print("You chose tags.")
    elif type == "search":
        addURL = f'search/{target_brain}?queryText={search}&maxResults={results}&onlySearchThoughtNames={nameOnly}'
        if verbose: print("You chose search.")
    elif type == "graph":
        addURL = f'thoughts/{target_brain}/{target}/graph'
        if verbose: print("You chose graph")
    elif type == "getNamedItem":
        addURL = f'thoughts/{target_brain}?nameExact={search}'
        if verbose: print("You chose Get Named Item")
    elif type == "getNoteMarkdown":
        addURL = f'notes/{target_brain}/{target}'
        if verbose: print("You chose Get Note in markdown")
    elif type == "getNoteHTML":
        addURL = f'notes/{target_brain}/{target}/html'
        if verbose: print("You chose Get Note in markdown")
    elif type == "getNoteText":
        addURL = f'notes/{target_brain}/{target}/text'
        if verbose: print("You chose Get Note in markdown")
    else:
        print("Invalid Type.")
        raise ValueError(f"Unknown request type: {type}")
    url = baseUrl + addURL
    if verbose: print(url)
    try:
        return requests.get(url, headers=get_headers)
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def createThought(thought):
    post_headers = {"Content-Type": "application/json-patch+json", 'Authorization': f'Bearer {apiKey}', 'accept': 'text/plain', 'Krang-Version': '2020-06-10'}
    addURL = ""
    addURL = f'thoughts/{target_brain}'
    url = baseUrl + addURL
    if verbose:  print(url)
    if verbose: print(thought)
    try:
        return requests.post(url, json=thought, headers=post_headers)
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None


def addNote(thought, note):
    post_headers = {"Content-Type": "application/json-patch+json", 'Authorization': f'Bearer {apiKey}', 'accept': '*/*', 'Krang-Version': '2020-06-10'}
    addURL = ""
    addURL = f'notes/{target_brain}/{thought}/update'
    url = baseUrl + addURL
    if verbose: print(url)
    if verbose: print(thought)
    note_obj = {"markdown": note}
    if verbose: print(note_obj)
    try:
        return requests.post(url, json=note_obj, headers=post_headers)  
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def addURL(thought, link, name):
    post_headers = {'Authorization': f'Bearer {apiKey}', 'accept': '*/*', 'Krang-Version': '2020-06-10'}
    addURL = ""
    safe_link = quote(link)
    safe_name = quote(name)
    addURL = f'attachments/{target_brain}/{thought}/url?url={safe_link}&name={safe_name}'
    url = baseUrl + addURL
    if verbose: print(url)
    if verbose: print(thought)
    try:
        return requests.post(url, headers=post_headers) 
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def addAttachment(thought, file_path):
    post_headers = {'Authorization': f'Bearer {apiKey}', 'accept': '*/*', 'Krang-Version': '2020-06-10'}
    addURL = ""
    addURL = f'attachments/{target_brain}/{thought}/file'
    url = baseUrl + addURL

    if verbose: print(url)
    if verbose: print(thought)
    if verbose: print(file_path)

    with open(file_path, 'rb') as fp:
        file_data = fp.read()
        files = {'file': (file_path, file_data)}
        try:
            return requests.post(url, headers=post_headers, files=files)
        except requests.RequestException as e:
            print(f"Request failed: {e}")
            return None
    
def addLink(from_id, to_id, relation):
    post_headers = {"Content-Type": "application/json-patch+json", 'Authorization': f'Bearer {apiKey}', 'accept': '*/*', 'Krang-Version': '2020-06-10'}
    addURL = ""

    addURL = f'links/{target_brain}'
    url = baseUrl + addURL
    if verbose: print(url)
    data = {
        "thoughtIdA": from_id,
        "thoughtIdB": to_id,
        "relation": relation
    }
    if verbose: print(data)
    try:
        return requests.post(url=url, json=data, headers=post_headers) 
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None


    

def process_response(response):
    global homeThought
    if response.status_code == 200:
        if verbose: print("Request was successful")
        # print(response.text)
        try:
            data = response.json()
            if "homeThoughtId" in data:
                homeThought = data["homeThoughtId"]
                print(f"Home Thought: {homeThought}")
            return True
        except (JSONDecodeError, KeyError) as e:
            print(f"Failed to parse response: {e}")
            return False
    elif response.status_code == 500:
        print("Request failed with status code:", response.status_code, ' Either there were no results, or your request was malformed.')
        print(response.text)
        return False
    elif response.status_code == 401:
        print("Request failed with status code:", response.status_code, ' There was a problem with your API key.')
        print(response.text)
        return False
    elif response.status_code == 404:
        print("Request failed with status code:", response.status_code , ". Please try again later.")
        print(response.text)
        return False
    else:
        print("Request failed with status code:", response.status_code)
        return False

In [ ]:
# List Brains

response = get("brainlist")
if process_response(response):
     for hit in response.json():
            print(hit)
    

In [ ]:
# BrainScan
response = get("brainscan")
if process_response(response):
    print(response.json())

In [ ]:
# List Types
response = get("types")
if process_response(response):
    print(response.json())


In [ ]:
# List Tags
response = get("tags")
if process_response(response):
    for hit in response.json():
        if hit['acType'] != 1:
            print(hit)

In [ ]:
#Search
response = get("search", search="Brain", results=10, nameOnly=True)
if process_response(response):
    print(f'{len(response.json())} results found.\n')
    for hit in response.json():
        print(hit)
        if hit['sourceThought']['acType'] != 1:
            print(hit['sourceThought'])

In [ ]:
#Graph
response = get("graph", target='af57d1bd-2e52-5c37-9e57-4c85aec40a65')
if process_response(response):
    for item in response.json():
        print(f'----{item}----')
        if type(item) == list:
            for hit in item:
                print(hit)
        else:
            print(response.json()[item])

In [ ]:
#Name Exact
response = get("getNamedItem", search='The Tempest')
if process_response(response):
    print(response.json())

In [ ]:
# Get HTML Note
response = get("getNoteHTML", target='af57d1bd-2e52-5c37-9e57-4c85aec40a65')
if process_response(response):
    for key, value in response.json().items():
        print(f'----{key}----')
        if isinstance(value, list):
            for hit in value:
                print(hit)
        else:
            print(value)

In [ ]:
# Get Markdown Note
response = get("getNoteMarkdown", target='af57d1bd-2e52-5c37-9e57-4c85aec40a65')
if process_response(response):
    for key, value in response.json().items():
        print(f'----{key}----')
        if isinstance(value, list):
            for hit in value:
                print(hit)
        else:
            print(value)


In [ ]:
# Get Text Note
response = get("getNoteText", target='af57d1bd-2e52-5c37-9e57-4c85aec40a65')
if process_response(response):
    for key, value in response.json().items():
        print(f'----{key}----')
        if isinstance(value, list):
            for hit in value:
                print(hit)
        else:
            print(value)

## Advanced Usage
Here I show a possible activity after extracting a text. I generated a \[linguistically shallow\] syntax tree with the text.

In [ ]:
# pip install nltk
# pip install svgling

import nltk
from nltk.corpus import treebank_chunk
from nltk import tokenize
from nltk import tag
from nltk import chunk
from IPython.display import display


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

text = ""
target_brain = '1c9d0bdb-8e55-43b2-ac53-959e4c2c8864'
response = get("getNoteText", target='af57d1bd-2e52-5c37-9e57-4c85aec40a65')

text = response.json()['text']
lines = text.split('\n')
for line in lines:
    if 'English' in line:
        sents = tokenize.sent_tokenize(line)
        sent = tokenize.word_tokenize(sents[0])
        tagged_sent = tag.pos_tag(sent)
        print(tagged_sent)
        tree = chunk.ne_chunk(tagged_sent)
        display(tree)
